In [ ]:
import os
import copy
from coffea import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd
import uproot3
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise
        
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)
        
maindirectory = os.getcwd() 
print(maindirectory)

In [ ]:
dir1 = 'CoffeaOutputsForCombine/Coffea_FirstRun/'
dir2 = 'CoffeaOutputsForNoSelectionTest/'
btagDir = ''
yearDir = '2016/'
APVDir = 'noAPV/'
od = ''
if btagDir == '':
    od = '_oldANdisc'

In [ ]:
JetHT2016_str = 'TTbarRes_0l_JetHT2016_Data'
TTbar_str = 'TTbarRes_0l_UL16postVFP_TTbar'

In [ ]:
JetHT2016_unweighted = util.load(dir2 + 'JetHT/' + btagDir + yearDir + JetHT2016_str + '_NoSelectionTest' + od + '.coffea')
TTbar_unweighted = util.load(dir2 + 'TT/' + btagDir + yearDir + APVDir + TTbar_str + '_NoSelectionTest' + od + '.coffea')

In [ ]:
outputs_unweighted =     {'JetHT 2016': JetHT2016_unweighted,
                          'TTbar': TTbar_unweighted
                         }

In [ ]:
for name,output in outputs_unweighted.items(): 
    print("-------Unweighted " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12f' % (i,j) )

In [ ]:
SaveDirectory_temp = maindirectory + '/TemporaryDirectory/NoAppliedMCWeights/' + btagDir + yearDir + APVDir # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory_temp) # no need to create the directory several times if it exists already

kin = ['pt', 'eta', 'phi', 'mass']
jet = ['ak8', 'ak4', 'sub']

stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_jet_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

for j in jet:
    for k in kin:
        
        name = j + 'jet' + k
        
        plt.rcParams.update({
            'font.size': 14,
            'axes.titlesize': 18,
            'axes.labelsize': 18,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12
            })
        fig, (ax, bx, cx) = plt.subplots(
            nrows=1,
            ncols=3,
            figsize=(20, 8),
            sharex=True,
            sharey='row'
        )
        
        TTbar_hist = TTbar_unweighted[name].integrate('dataset', TTbar_str[12:])
        JetHT_hist = JetHT2016_unweighted[name].integrate('dataset', JetHT2016_str[12:])

        TTbarPlot = hist.plot1d(TTbar_hist, ax=ax, clear=True,
                            fill_opts=stack_ttbar_opts,
                            error_opts=stack_error_opts)

        JetHTPlot = hist.plot1d(JetHT_hist, ax=bx, clear=True,
                            fill_opts=stack_jet_opts,
                            error_opts=stack_error_opts)

        TTbarPlot2 = hist.plot1d(TTbar_hist, ax=cx, clear=True,
                            line_opts={'color':'red'},
                            error_opts=stack_error_opts)
        JetHTPlot2 = hist.plot1d(JetHT_hist, ax=cx, clear=False,
                            line_opts={'color':'orange'},
                            error_opts=stack_error_opts)

        ax.autoscale('y')
        bx.set_ylabel('')
        cx.set_ylabel('')

        ax.yaxis.grid()
        bx.yaxis.grid()
        cx.yaxis.grid()

        bx.set_title(r'No Selections Applied', fontsize='xx-large', loc = 'center', y=1.05)

        lega = ax.legend(title='v9 Dataset', title_fontsize='x-large',
                      labels=['TTbar'], fontsize='medium')
        legb = bx.legend(title='v9 Dataset', title_fontsize='x-large',
                      labels=['JetHT'], fontsize='medium')
        legc = cx.legend(title='v9 Datasets', title_fontsize='x-large',
                      labels=['TTbar', 'JetHT'], fontsize='medium')

        if 'ak8jetpt' in name:
            ax.set_xlim(0,1000)
        elif 'ak8jetmass' in name:
            ax.set_xlim(0,300)
        elif 'ak4jetmass' in name:
            ax.set_xlim(0,80)
        elif 'ak4jetpt' in name:
            ax.set_xlim(0,600)
        elif 'subjetpt' in name:
            ax.set_xlim(0,800)
        elif 'subjetmass' in name:
            ax.set_xlim(0,150)
            
        coffee = plt.text(3.20, 1.00, u"☕",
                          fontsize=50,
                          horizontalalignment='left',
                          verticalalignment='bottom',
                          transform=ax.transAxes
                         )

        filename_temp = 'ak8jetpt.png'
        # plt.savefig(SaveDirectory_temp+filename_temp, bbox_inches="tight")